<a href="https://colab.research.google.com/github/SridharSeshadri56/Decision_Models/blob/main/pyomoFixedChargeProblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Sets up Fixed Charge problem from set of supply nodes to demand nodes 
# Cost of shipping per unit from supply to demand nodes, supply at supply nodes, 
# demand at demand nodes are given, capacity and fixed cost of capacity at supply nodes 

In [ ]:
pip install pyomo  #Installs the modeling language called pyomo

In [ ]:
# The GLPK (GNU Linear Programming Kit) package is intended for solving large-scale linear programming (LP), 
# mixed integer programming (MIP), and other related problems. It is a set of routines written in ANSI C and 
# organized in the form of a callable library.
!apt-get install -y -qq glpk-utils  #Installs the optimization engine called glpk.


In [8]:
from pyomo.environ import *
import itertools                          # package helps create products of sets. here I use to create product of demand and supply sets

demand_points = ['A', 'B', 'C', 'D', 'O', 'P']    # This is the set of demand points
supply_points = ['L', 'H', 'S', 'M', 'W']                        # This is the set of supply points

demand = {'A': 10, 'B': 8, 'C': 14, 'D': 6, 'O':7, 'P':11}          # demand at demand points 
supply = {'L': 18, 'H': 24, 'S': 27, 'M':22, 'W':31}                # max supply capacity at supply points if open
fixed_cost = {'L': 7650, 'H': 3500, 'S': 5000, 'M':4100, 'W':2200}

flows_from_to_arcs = list(itertools.product(supply, demand))   # this creates a list with supply and demand point names
                                                               # this will become our set of variables
                                                               # we could loop over supply and demand and add to a list instead

cost_from_to = {('L','A'):1675, ('L','B'): 400, ('L','C'): 685, ('L', 'D'):1630, ('L','O'):1160, ('L','P'):3800,  \
                ('H','A'):1460, ('H','B'):1940, ('H','C'): 970, ('H', 'D'): 100, ('H','O'): 495, ('H','P'):1200,  \
                ('S','A'):1925, ('S','B'):2400, ('S','C'):1425, ('S', 'D'): 500, ('S','O'): 950, ('S','P'): 800,  \
                ('M','A'): 380, ('M','B'):1355, ('M','C'): 543, ('M', 'D'):1045, ('M','O'): 665, ('M','P'):2321,  \
                ('W','A'): 922, ('W','B'):1646, ('W','C'): 700, ('W', 'D'): 508, ('W','O'): 311, ('W','P'):1797
                 }       
                       # cost of ship one unit from- to

constraints = {'supply_constraint', 'demand_constraint'}           # The two sets of constraints

model = ConcreteModel(name = "(Model2)")                            # Same as previous
model.x = Var( flows_from_to_arcs, within= NonNegativeReals )       # Decision variables are the flows from - to
model.y = Var( supply_points, within = Binary)
model.value = Objective(                                            # Objective
expr = sum( cost_from_to[i]*model.x[i] for i in flows_from_to_arcs) + sum(fixed_cost[i]*model.y[i] for i in supply_points), sense = minimize )  
# Minimize total transportation cost plus fixed charge if supply is open

# This defines a rule called demand must be met. 

def demand_must_be_met_rule(m,c):
    return sum(m.x[i,c] for i in supply_points) == demand [c]    # sums supply to demand point c. Note the syntax == for saying equal to

# This defines a rule to make sure do not exceed supply constraints

def supply_must__not_be_exceeded_rule(m,c):
    return sum(m.x[c,i] for i in demand_points) <= supply [c] * model.y[c]   
    # sums from supply point c to demand points. Right hand side equals capacity if plant open else is equal to zero

# This defines in our model the constraints! Note that we simply pass the set of constraints and the rule. It does the rest.
# model is by default when we call (recall model can be renamed as you like )
model.demand_constraint = Constraint(demand_points, rule = demand_must_be_met_rule)   # applies to each demand point

model.supply_constraint = Constraint(supply_points, rule = supply_must__not_be_exceeded_rule)  # applies to each supply point

opt = SolverFactory('glpk')           # same as before

model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)   # same as before
results = opt.solve(model, tee= True)                 # same as before

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmp1_ja5z24.glpk.raw --wglp /tmp/tmpvily33hw.glpk.glp --cpxlp
 /tmp/tmp9oti5iu5.pyomo.lp
Reading problem data from '/tmp/tmp9oti5iu5.pyomo.lp'...
/tmp/tmp9oti5iu5.pyomo.lp:181: warning: lower bound of variable 'x31' redefined
/tmp/tmp9oti5iu5.pyomo.lp:181: warning: upper bound of variable 'x31' redefined
12 rows, 36 columns, 66 non-zeros
5 integer variables, all of which are binary
186 lines were read
Writing problem data to '/tmp/tmpvily33hw.glpk.glp'...
195 lines were written
GLPK Integer Optimizer, v4.65
12 rows, 36 columns, 66 non-zeros
5 integer variables, all of which are binary
Preprocessing...
11 rows, 35 columns, 65 non-zeros
5 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  3.100e+01  ratio =  3.100e+01
GM: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
EQ: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
2N

In [ ]:
model.pprint()

SINGLE SOURCE: EACH DEMAND POINT CAN BE SERVED BY ONLY ONE SOURCE.
One can easily modify this formulation to make sure each demand point is served by a single source. The idea is to replace flows with whether demand is served by a supply (binary). Then to multiply the decision by the demand. See modification below 

In [9]:
demand_points = ['A', 'B', 'C', 'D', 'O', 'P']    # This is the set of demand points
supply_points = ['L', 'H', 'S', 'M', 'W']                        # This is the set of supply points

demand = {'A': 10, 'B': 8, 'C': 14, 'D': 6, 'O':7, 'P':11}          # demand at demand points 
supply = {'L': 18, 'H': 24, 'S': 27, 'M':22, 'W':31}                # max supply capacity at supply points if open
fixed_cost = {'L': 7650, 'H': 3500, 'S': 5000, 'M':4100, 'W':2200}

flows_from_to_arcs = list(itertools.product(supply, demand))   # this creates a list with supply and demand point names
                                                               # this will become our set of variables
                                                               # we could loop over supply and demand and add to a list instead

cost_from_to = {('L','A'):1675, ('L','B'): 400, ('L','C'): 685, ('L', 'D'):1630, ('L','O'):1160, ('L','P'):3800,  \
                ('H','A'):1460, ('H','B'):1940, ('H','C'): 970, ('H', 'D'): 100, ('H','O'): 495, ('H','P'):1200,  \
                ('S','A'):1925, ('S','B'):2400, ('S','C'):1425, ('S', 'D'): 500, ('S','O'): 950, ('S','P'): 800,  \
                ('M','A'): 380, ('M','B'):1355, ('M','C'): 543, ('M', 'D'):1045, ('M','O'): 665, ('M','P'):2321,  \
                ('W','A'): 922, ('W','B'):1646, ('W','C'): 700, ('W', 'D'): 508, ('W','O'): 311, ('W','P'):1797
                 }       
                       # cost of ship one unit from- to

constraints = {'supply_constraint', 'demand_constraint'}           # The two sets of constraints

model = ConcreteModel(name = "(Model2)")                            # Same as previous
model.x = Var( flows_from_to_arcs, within= Binary )                 # Decision variables are whether to serve demand point from supply point
model.y = Var( supply_points, within = Binary)
model.value = Objective(                                            # Objective
expr = sum( cost_from_to[i,j]*model.x[i,j]*demand[j] for (i,j) in flows_from_to_arcs) + sum(fixed_cost[i]*model.y[i] for i in supply_points), sense = minimize )  
# Minimize total transportation cost plus fixed charge if supply is open
# Note transportation cost = whether supply * cost of supply * demand

# This defines a rule called demand must be met. 

def demand_must_be_met_rule(m,c):
    return sum(m.x[i,c] for i in supply_points) == 1    # sums supply to demand point c. Note the syntax == for saying equal to
    # This rule says each demand must be served by one supply at most

# This defines a rule to make sure do not exceed supply constraints

def supply_must__not_be_exceeded_rule(m,c):
    return sum(m.x[c,i]*demand[i] for i in demand_points) <= supply [c] * model.y[c]   
    # sums from supply point c to demand points. Right hand side equals capacity if plant open else is equal to zero
    # Note we multiply by demand because m.x are zero-one variables

# This defines in our model the constraints! Note that we simply pass the set of constraints and the rule. It does the rest.
# model is by default when we call (recall model can be renamed as you like )
model.demand_constraint = Constraint(demand_points, rule = demand_must_be_met_rule)   # applies to each demand point

model.supply_constraint = Constraint(supply_points, rule = supply_must__not_be_exceeded_rule)  # applies to each supply point

opt = SolverFactory('glpk')           # same as before

model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)   # same as before
results = opt.solve(model, tee= True)                 # same as before

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpk_8nj2n4.glpk.raw --wglp /tmp/tmpjtfz97lo.glpk.glp --cpxlp
 /tmp/tmp4ef8rbz4.pyomo.lp
Reading problem data from '/tmp/tmp4ef8rbz4.pyomo.lp'...
/tmp/tmp4ef8rbz4.pyomo.lp:181: warning: lower bound of variable 'x1' redefined
/tmp/tmp4ef8rbz4.pyomo.lp:181: warning: upper bound of variable 'x1' redefined
12 rows, 36 columns, 66 non-zeros
35 integer variables, all of which are binary
216 lines were read
Writing problem data to '/tmp/tmpjtfz97lo.glpk.glp'...
165 lines were written
GLPK Integer Optimizer, v4.65
12 rows, 36 columns, 66 non-zeros
35 integer variables, all of which are binary
Preprocessing...
11 rows, 35 columns, 65 non-zeros
35 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  3.100e+01  ratio =  3.100e+01
GM: min|aij| =  9.656e-01  max|aij| =  1.036e+00  ratio =  1.072e+00
EQ: min|aij| =  9.484e-01  max|aij| =  1.000e+00  ratio =  1.054e+00
2

In [10]:
model.pprint()

4 Set Declarations
    demand_constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {'A', 'B', 'C', 'D', 'O', 'P'}
    supply_constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'L', 'H', 'S', 'M', 'W'}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     2 :    Any :   30 : {('L', 'A'), ('L', 'B'), ('L', 'C'), ('L', 'D'), ('L', 'O'), ('L', 'P'), ('H', 'A'), ('H', 'B'), ('H', 'C'), ('H', 'D'), ('H', 'O'), ('H', 'P'), ('S', 'A'), ('S', 'B'), ('S', 'C'), ('S', 'D'), ('S', 'O'), ('S', 'P'), ('M', 'A'), ('M', 'B'), ('M', 'C'), ('M', 'D'), ('M', 'O'), ('M', 'P'), ('W', 'A'), ('W', 'B'), ('W', 'C'), ('W', 'D'), ('W', 'O'), ('W', 'P')}
    y_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :    